# **1. Data Wrangling**

**Data Wrangling Summary**

In the data wrangling section of this project, I focus on collecting, associating, defining, and cleaning the data to prepare it for further analysis. Here is a summary of the subsection within the data wrangling process:

**1. Data Collection:**
   - Gather data from two sources: ball-by-ball match data from Kaggle and salary data from the Cricmetric website.
   - The match data provides detailed information about each IPL match from 2008 to 2021, while the salary data contains player salary information.
   
**2. Data Association:**
   - Restrict analysis to the year for which we have salary data, excluding IPL 2021-22 from the ball-by-ball match data.
   - Associate player names across the two datasets by identifying common identifiers, such as the full first name in the salary data.
   
**3. Data Definition and Cleaning:**
   - The goal is to calculate the correlation between various features and player salary.
   - Derive player statistics focusing on batting and bowling performances:
     - Batting statistics include total runs, average, strike rate, 50s, 100s, etc.
     - Bowling statistics include total wickets, average, economy rate, strike rate, 3 wicket hauls, 5 wicket hauls, 4s, 6s, etc.
   - Consider general player statistics, such as the number of seasons played, total matches played in the previous season, previous-year team, and previous-year salary.
   - Merge the various dataframes by player role, export, and save.
   
This data wrangling process allows us to transform and organize the data to facilitate correlation analysis and further exploration of the relationship between player performance and salary in the IPL.

## Part 0: Import Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from ipywidgets import interact

## Part 1: Data Collection

There are two places from which I will collect the data:
1. Ball-by-ball match data from https://www.kaggle.com/datasets/vora1011/ipl-2008-to-2021-all-match-dataset
2. Salary data from http://www.cricmetric.com/ipl/salary/

### **(I) IPL ball-by ball data from Kaggle**

For this section of the project, I can simply use the Kaggle API.

In [5]:
# configure the kaggle.json file which contains my API token information.
!chmod 600 './kaggle/.kaggle/kaggle.json'

In [6]:
# Use the kaggle python library to access the API to download the dataset from Kaggle
!kaggle datasets download -d vora1011/ipl-2008-to-2021-all-match-dataset -p ~/Downloads

ipl-2008-to-2021-all-match-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
# relocate the download to my removable volume
!cp '~/Downloads/ipl-2008-to-2021-all-match-dataset.zip' './ipl-player-salary/IPLData'


In [8]:
# unzip the file
!unzip './ipl-player-salary/IPLData/ipl-2008-to-2021-all-match-dataset.zip' -d './ipl-player-salary/IPLData/'

Archive:  /Volumes/My Passport for Mac/DataScienceCap2/IPLData/ipl-2008-to-2021-all-match-dataset.zip
replace /Volumes/My Passport for Mac/DataScienceCap2/IPLData/IPL_Ball_by_Ball_2008_2022.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
# save the data in the pandas dataframes ball_data_full and match_data_full
path_to_ball_data = "./IPLData/IPL_Ball_by_Ball_2008_2022.csv"
path_to_match_data = "./IPLData/IPL_Matches_2008_2022.csv"

df_ball_data_full = pd.read_csv(path_to_ball_data)

# now we need to order this data by the player, since that is what we care about ultimately.
df_ball_data_full = df_ball_data_full.sort_values('batter')

df_match_data_full = pd.read_csv(path_to_match_data)

### **(II) Player Salary Data from CricMetric**

Since CricMetric doesn't have an API, I will have to download the data directly from their website: http://www.cricmetric.com/ipl/salary/ 

I have stored the data in my removable volume.

In [8]:
def salary_data(year):
    
    '''Function to return the salary data of the inputted year as a pandas dataframe.'''

    base_string = "./IPLData/"
    added_string = "ipl_salary_" + str(year) + ".csv"
    basic_df = pd.read_csv(base_string + added_string)

    basic_df =  basic_df.dropna(subset=['Player.1'])
    basic_df = basic_df.assign(year=year)

    return basic_df

In [9]:
dfs = []
for year in range(2008, 2022):
    df = salary_data(year)
    df['year'] = year
    dfs.append(df)

full_salary_df = pd.concat(dfs, ignore_index=True)
full_salary_df = full_salary_df.sort_values(['Player', 'year'])

## Part 2: Data Association

### **(I) Restrict the ball-by-ball data**

The first thing we need to do is to restrict ourselves to the years for which we have the salary data. This means we basically ignore the data from IPL 2021-22 in the dataframe `df_ball_data_full`.

In [10]:
merged_df = pd.merge(df_ball_data_full, df_match_data_full[['ID', 'Season']], on='ID')
merged_df.dtypes

merged_df['Season'] = merged_df['Season'].replace('2007/08', '2008')
merged_df['Season'] = merged_df['Season'].replace('2009/10', '2010')
merged_df['Season'] = merged_df['Season'].replace('2020/21', '2020')
merged_df['Season'] = merged_df['Season'].astype(int)

df_ball_data_restricted = merged_df[(merged_df['Season'] >= 2008) & (merged_df['Season'] <= 2020)].sort_values('batter')

### **(II) Player Names**

The second thing we need to do is to associate the names in the dataframes. As we can see from the dataframes `full_salary_df` and `df_ball_data_full`, we need a way to associate the names across the two dataframes. Clearly, in `full_salary_df`, the full first name of the player is present only when the player name as represented by 'First Initial' + 'Last Name' is not unique. This should give us a method to associate the names across the dataframes.

In [11]:
unique_players_salary = pd.DataFrame(full_salary_df.Player.unique())
unique_players_bat = pd.DataFrame(df_ball_data_restricted.batter.unique())

accounted_players_bat = []

for player in unique_players_bat.values:
    if player in unique_players_salary.values:
        accounted_players_bat.append(player[0])
    else:
        pass

print(len(accounted_players_bat))

407


In [12]:
# First let us check that there are no duplicate players in this list:
if len(accounted_players_bat) == len(set(accounted_players_bat)):
    print("There are no duplicates in accounted_players.")
else:
    print("There are duplicates in accounted_players.")

There are no duplicates in accounted_players.


Having verified that there are no duplicate players in this list, and given that we have 407 players, for this project, I will restrict my analysis to these 407 players. This will restrict our view to players who have faced at least one delivery in the match.

## Part 3: Data Definition

### **(I) Player Statistics**

#### (A) Batting Statistics

In [26]:
def batting_data(player, year):

    batsman_data = df_ball_data_restricted[(df_ball_data_restricted['Season'] == year) &
                                           (df_ball_data_restricted['batter'] == player)]
    num_matches_played = batsman_data['ID'].nunique() 
    
    if num_matches_played != 0:

        total_runs = batsman_data['batsman_run'].sum()

        if batsman_data['isWicketDelivery'].sum() != 0:
            batting_avg = total_runs / batsman_data['isWicketDelivery'].sum()
        else:
            batting_avg = total_runs

        balls_faced = len(batsman_data)
        player_innings = batsman_data.groupby('ID')['batsman_run'].sum()

        num4s = len(batsman_data[batsman_data['batsman_run'] == 4])
        num6s = len(batsman_data[batsman_data['batsman_run'] == 6])
        num_50 = sum((player_innings>=50)&(player_innings<100))
        num_100 = sum(player_innings>=100)
        
        if balls_faced != 0:
            strike_rate = (total_runs / balls_faced) * 100
        else:
            strike_rate = 0

        # Create a dataframe with all the calculated data for the player in the given phase
        batting_df = pd.DataFrame({'player': [player], 'season': [year], 'balls_faced': [balls_faced],
                                   'total_runs': [total_runs], 'batting_avg': [batting_avg],
                                   'strike_rate': [strike_rate], '50s': [num_50], '100s': [num_100], '4s': [num4s], '6s': [num6s]})
        return batting_df
    else:
        return pd.DataFrame()

In [28]:
batting = []

for year in range(2008, 2021):    
    unique_batters = list(df_ball_data_restricted[df_ball_data_restricted['Season'] == year]['batter'].unique())
    for batter in unique_batters:
        batting.append(batting_data(batter, year))

batting_comb = pd.concat(batting, ignore_index=True)
batting_comb = batting_comb.sort_values(['player', 'season', 'total_runs'], ascending=[True, True, False])

#### (B) Bowling Statistics

In [39]:
def bowling_data(player, year):
    bowler_data = df_ball_data_restricted[(df_ball_data_restricted['Season'] == year) &
                                           (df_ball_data_restricted['bowler'] == player)]
    num_matches_played = bowler_data['ID'].nunique() 
    
    if num_matches_played != 0:
        total_runs = bowler_data['total_run'].sum()
        total_wickets = bowler_data['isWicketDelivery'].sum()
        balls_bowled = len(bowler_data)
        num4s = len(bowler_data[bowler_data['batsman_run'] == 4])
        num6s = len(bowler_data[bowler_data['batsman_run'] == 6])
        num_dots = len(bowler_data[bowler_data['total_run'] == 0])

        bowler_innings = bowler_data.groupby('ID')['isWicketDelivery'].sum()
        num_3whs = sum((bowler_innings>=3)&(bowler_innings<5))
        num_5whs = sum(bowler_innings>=5)
       
        if total_wickets != 0 and balls_bowled != 0:
            bowling_avg = total_runs / total_wickets
            strike_rate = balls_bowled / total_wickets
            economy = (total_runs / balls_bowled) * 6
        else:
            bowling_avg = total_runs
            strike_rate = balls_bowled
            economy = bowling_avg/strike_rate

        # Create a dataframe with all the calculated data for the player in the given phase
        bowling_df = pd.DataFrame({'player': [player], 'season': [year], 'balls_bowled': [balls_bowled], 'total_runs': [total_runs],
                                   'total_wickets': [total_wickets], 'bowling_avg': [bowling_avg], 'economy': [economy],
                                   'strike_rate': [strike_rate], '3whs': [num_3whs], '5whs': [num_5whs], 'dots': [num_dots],'4s': [num4s], '6s': [num6s]})
        return bowling_df
    else:
        return pd.DataFrame()

In [40]:
bowling = []

for year in range(2008, 2021):    
    unique_bowlers = list(df_ball_data_restricted[df_ball_data_restricted['Season'] == year]['bowler'].unique())
    for bowler in unique_bowlers:
        bowling.append(bowling_data(bowler, year))

bowling_comb = pd.concat(bowling, ignore_index=True)
bowling_comb = bowling_comb.sort_values(['player', 'season', 'total_wickets'], ascending=[True, True, False])

#### (C) General Player Statistics

In [51]:
df_match_data_full['Season'] = df_match_data_full['Season'].replace('2007/08', '2008')
df_match_data_full['Season'] = df_match_data_full['Season'].replace('2009/10', '2010')
df_match_data_full['Season'] = df_match_data_full['Season'].replace('2020/21', '2020')
df_match_data_full['Season'] = df_match_data_full['Season'].astype(int)

df_match_data_restricted = df_match_data_full[df_match_data_full['Season'].isin(range(2008,2021))]
batter_list = list(batting_comb['player'].unique())
player_match_counts_df = pd.DataFrame(columns=['Player', 'Season', 'MatchCount'])
for season in range(2008, 2021):
    for player in batter_list:
        # Filter the match data to find matches where the player played in the current season
        matches_played = df_match_data_restricted[(df_match_data_restricted['Season'] == season) &
                                                  ((df_match_data_restricted['Team1Players'].apply(lambda x: player in x)) |
                                                   (df_match_data_restricted['Team2Players'].apply(lambda x: player in x)))]
        match_count = len(matches_played)
        player_match_counts_df = pd.concat([player_match_counts_df[player_match_counts_df['MatchCount'] > 0],
                                            pd.DataFrame({'Player': player, 'Season': season, 'MatchCount': match_count}, index=[0])],
                                           ignore_index=True)

player_match_counts_df = player_match_counts_df.sort_values(['Player', 'Season'])
player_match_counts_df['SeasonCount'] = player_match_counts_df.groupby('Player').cumcount() + 1
player_match_counts_df = player_match_counts_df[player_match_counts_df['MatchCount'] > 0]

In [53]:
# Merge the player match counts dataframe and the full salary dataframe based on matching player and season
general_comb = pd.merge(player_match_counts_df, full_salary_df, left_on=['Player', 'Season'], right_on=['Player', 'year'], how='inner')
general_comb = general_comb[['Player', 'Season', 'MatchCount', 'SeasonCount', 'Team', 'Salary']]
general_comb['Salary'] = general_comb['Salary'].str.replace('$', '').str.replace(',', '').astype(int)
general_comb = general_comb.rename(columns={'Salary': 'SalaryUSD'})

/var/folders/3s/6yq71yf16yq1y9c5zlss3syc0000gn/T/ipykernel_13869/392744986.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  general_comb['Salary'] = general_comb['Salary'].str.replace('$', '').str.replace(',', '').astype(int)


I have scraped the rest of the data for the general player statistics, the role and the country of origin, from the CricMetric site. I then stored it in the file `player_country_role.txt`. The details of this can be found in the `WikiScrape.ipynb` file. 

In [24]:
# Apply the country_and_role function to each player name in the Player column
len(list(general_comb['Player'].unique()))
new_player_list = list(general_comb['Player'].unique())
pkl_path = '../IPLData/new_player_list.pkl'

# Save the list to a file
with open(pkl_path, 'wb') as f:
    pickle.dump(new_player_list, f)

In [55]:
# define a filepath
filepath = '../IPLData/player_country_role.txt'

In [56]:
# with the updated information, reset the dataframe:
player_country_role_df = pd.read_csv(filepath, sep='\t')
player_country_role_df.head()

,Player,Role,Country
0,Mohammad Nabi,Allrounder,Afghanistan
1,Mujeeb Ur Rahman,Bowler,Afghanistan
2,Rashid Khan,Bowler,Afghanistan
3,A Zampa,Bowler,Australia
4,AJ Tye,Bowler,Australia


In [57]:
# Merge the dataframes
merged_general_comb = general_comb.merge(player_country_role_df, on='Player', how='left')
merged_general_comb = merged_general_comb.sort_values('Player')

#### (D) Review

I will use this section to take a look at the three dataframes that we have created.

In [58]:
batting_comb.head()

,player,season,balls_faced,total_runs,batting_avg,strike_rate,50s,100s,4s,6s
648,A Ashish Reddy,2012,30,35,8.750000,116.666667,0,0,3,1
818,A Ashish Reddy,2013,90,125,20.833333,138.888889,0,0,8,5
1123,A Ashish Reddy,2015,46,73,24.333333,158.695652,0,0,3,5
1252,A Ashish Reddy,2016,30,47,23.500000,156.666667,0,0,2,4
649,A Chandila,2012,1,0,0.000000,0.000000,0,0,0,0


In [59]:
bowling_comb.head()

,player,season,balls_bowled,total_runs,total_wickets,bowling_avg,economy,strike_rate,3whs,5whs,dots,4s,6s
493,A Ashish Reddy,2012,169,238,11,21.636364,8.449704,15.363636,1,0,55,14,12
648,A Ashish Reddy,2013,41,71,3,23.666667,10.390244,13.666667,0,0,10,7,3
868,A Ashish Reddy,2015,37,51,4,12.750000,8.270270,9.250000,0,0,9,1,3
980,A Ashish Reddy,2016,23,40,1,40.000000,10.434783,23.000000,0,0,5,4,2
512,A Chandila,2012,84,87,5,17.400000,6.214286,16.800000,1,0,39,3,6


In [60]:
merged_general_comb.head()

,Player,Season,MatchCount,SeasonCount,Team,SalaryUSD,Role,Country
0,A Ashish Reddy,2015,6,3,Sunrisers Hyderabad,33000,Bowler,Uncapped
1,A Ashish Reddy,2016,4,4,Sunrisers Hyderabad,33000,Bowler,Uncapped
2,A Choudhary,2017,5,1,Royal Challengers Bangalore,299000,Bowler,Uncapped
3,A Dananjaya,2018,1,1,Mumbai Indians,78130,Allrounder,Sri Lanka
4,A Flintoff,2009,3,1,Chennai Super Kings,1550000,Allrounder,England


In [62]:
batting_comb[batting_comb['player']=='SM Curran']

,player,season,balls_faced,total_runs,batting_avg,strike_rate,50s,100s,4s,6s
1787,SM Curran,2019,55,95,23.75,172.727273,1,0,13,3
1917,SM Curran,2020,145,186,23.25,128.275862,1,0,12,12


In [63]:
bowling_comb[bowling_comb['player']=='SM Curran']

,player,season,balls_bowled,total_runs,total_wickets,bowling_avg,economy,strike_rate,3whs,5whs,dots,4s,6s
1265,SM Curran,2019,207,325,11,29.545455,9.420290,18.818182,2,0,58,31,12
1392,SM Curran,2020,265,348,14,24.857143,7.879245,18.928571,2,0,87,36,8


In [65]:
merged_general_comb = merged_general_comb[['Player', 'Country', 'Role', 'Season', 'Team', 'SalaryUSD', 'MatchCount', 'SeasonCount']]
merged_general_comb[merged_general_comb['Player']=='SM Curran'].sort_values('Season')

,Player,Country,Role,Season,Team,SalaryUSD,MatchCount,SeasonCount
1248,SM Curran,England,Allrounder,2019,Kings XI Punjab,1028600,9,1
1249,SM Curran,England,Allrounder,2020,Chennai Super Kings,785700,14,2


### **(III) Final Merging by Player Role**

In this section, I will merge the three dataframes by player role.

In [66]:
# First we filter merged_general_comb for any player role that includes some string 'role'
def filtered_merged_general_comb(role):
    return merged_general_comb[merged_general_comb['Role'].str.contains(role, case=False)]

In [69]:
# Check the number of common players
unique_batters_merged_general_comb = filtered_merged_general_comb('batsman')['Player'].unique()
unique_players_batting_comb = batting_comb['player'].unique()

common_players = np.intersect1d(unique_batters_merged_general_comb, unique_players_batting_comb)
count_common_players = len(common_players)
print(count_common_players)

146


In [70]:
# Check the number of common players
unique_bowlers_merged_general_comb = filtered_merged_general_comb('Bowler')['Player'].unique()
unique_players_bowling_comb = bowling_comb['player'].unique()

common_players = np.intersect1d(unique_bowlers_merged_general_comb, unique_players_bowling_comb)
count_common_players = len(common_players)
print(count_common_players)

153


In [71]:
# Check the number of common players
unique_allrounders_merged_general_comb = filtered_merged_general_comb('Allrounder')['Player'].unique()

common_players_bat = np.intersect1d(unique_allrounders_merged_general_comb, unique_players_batting_comb)
common_players_bowl = np.intersect1d(unique_allrounders_merged_general_comb, unique_players_bowling_comb)
count_bat = len(common_players_bat)
count_bowl = len(common_players_bowl)

common_players = np.intersect1d(np.intersect1d(unique_allrounders_merged_general_comb, unique_players_batting_comb), unique_players_bowling_comb)
count_common_players = np.intersect1d(common_players, common_players_bowl)

print(count_bat, count_bowl, len(common_players), len(count_common_players))
# From this, we can deduce that all the players in the bowling df with the role 'allrounder' will appear in the other dataframes

99 96 96 96


### (A) Batters

In [84]:
# Perform a left merge on 'Player' and 'player' columns, as well as 'Season'
batter_data = filtered_merged_general_comb('batsman').merge(batting_comb, left_on=['Player', 'Season'], right_on=['player', 'season'], how='left')
batter_data = batter_data.drop(['player', 'season'], axis=1).sort_values(['Player', 'Season'])

In [86]:
# Perform a left merge on 'Player' and 'player' columns for the bowling data
ar_data_bat = filtered_merged_general_comb('Allrounder').merge(batting_comb, left_on=['Player', 'Season'], right_on=['player', 'season'])
ar_data_bat = ar_data_bat.drop(['player', 'season'], axis=1).sort_values(['Player', 'Season'])

In [90]:
merged_batter_data = pd.concat([batter_data, ar_data_bat])

In [91]:
batter_pkl_path = '../FinalDFs/PostWrangling/batting_merged.pkl'
with open(batter_pkl_path, 'wb') as f:
    pickle.dump(merged_batter_data, f)

### (B) Bowlers

In [92]:
bowler_data = filtered_merged_general_comb('Bowler').merge(bowling_comb, left_on=['Player', 'Season'], right_on=['player', 'season'], how='left')
bowler_data = bowler_data.drop(['player', 'season'], axis=1).sort_values(['Player', 'Season'])

In [94]:
ar_data_bowl = filtered_merged_general_comb('Allrounder').merge(bowling_comb, left_on=['Player', 'Season'], right_on=['player', 'season'])
ar_data_bowl = ar_data_bowl.drop(['player', 'season'], axis=1).sort_values(['Player', 'Season'])

In [95]:
merged_bowler_data = pd.concat([bowler_data, ar_data_bowl])

In [97]:
bowler_pkl_path = '../FinalDFs/PostWrangling/bowling_merged.pkl'
with open(bowler_pkl_path, 'wb') as f:
    pickle.dump(merged_bowler_data, f)